In [48]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import scipy
import h5py
from scipy.stats import entropy
from scipy.spatial.distance import pdist, squareform
import scipy.cluster.hierarchy as _sch
from activ import load_data
from activ.readfile import load_preprocessed, TrackTBIFile, encode, impute
import pandas as pd
from math import ceil
from matplotlib import cm
from sklearn import preprocessing
from collections import Counter

In [4]:
subdata = load_data()
subdata_oc = subdata.outcomes
subdata_bm = subdata.biomarkers
oc_features = subdata.outcome_features
bm_features = subdata.biomarker_features
sub_features = np.hstack((subdata.biomarker_features, subdata.outcome_features))
ids = subdata.patient_ids

AttributeError: 'TrackTBIFile' object has no attribute 'shape'

In [5]:
fulldata = load_preprocessed()

In [50]:
fulldata.GCSMildModSevereRecode

BR-1001    0.0
BR-1003    0.0
BR-1004    0.0
BR-1006    0.0
BR-1007    0.0
BR-1008    0.0
BR-1009    0.0
BR-1010    0.0
BR-1011    0.0
BR-1012    0.0
BR-1013    0.0
BR-1014    0.0
BR-1015    0.0
BR-1016    0.0
BR-1017    0.0
BR-1018    0.0
BR-1019    0.0
BR-1020    0.0
BR-1021    0.0
BR-1023    0.0
BR-1024    0.0
BR-1025    0.0
BR-1026    0.0
BR-1027    0.0
BR-1028    0.0
BR-1029    0.0
BR-1030    0.0
BR-1031    0.0
BR-1032    2.0
BR-1033    0.0
          ... 
SF-1315    0.0
SF-1316    0.0
SF-1318    0.0
SF-1319    0.0
SF-1320    0.0
SF-1321    0.0
SF-1322    0.0
SF-1323    0.0
SF-1324    0.0
SF-1325    0.0
SF-1326    0.0
SF-1327    1.0
SF-1328    0.0
SF-1329    0.0
SF-1330    0.0
SF-1332    0.0
SF-1333    0.0
SF-1334    0.0
SF-1335    0.0
SF-1336    0.0
SF-1337    0.0
SF-1338    0.0
SF-1339    0.0
SF-1340    0.0
SF-1341    0.0
SF-1342    0.0
SF-1343    0.0
SF-1344    0.0
SF-1345    0.0
SF-1346    0.0
Name: GCSMildModSevereRecode, Length: 586, dtype: float64

In [27]:
gose1 = fulldata[fulldata.GOSE_OverallScore6M == 1.0]

In [28]:
gose1_ids = gose1.index.astype(str)

In [29]:
gose1_ids

Index(['BR-1043', 'BR-1078', 'PI-1014', 'PI-1020', 'PI-1051', 'PI-1052',
       'PI-1053', 'PI-1061', 'PI-1065', 'PI-1091', 'PI-1098', 'PI-1113',
       'PI-1131', 'PI-1137', 'PI-1145', 'PI-1161', 'PI-1175', 'SF-1009',
       'SF-1073', 'SF-1118', 'SF-1129', 'SF-1130', 'SF-1162', 'SF-1168',
       'SF-1211', 'SF-1267', 'SF-1271', 'SF-1316'],
      dtype='object')

In [43]:
for ii,f in enumerate(fulldata.columns):
    if 'EDArr' in f:
        print(f,ii)

EDArrSBP 148
EDArrDBP 149
EDArrHR 150
EDArrRespRate 151
EDArrRespRateType 152
EDArrTemp 153
EDArrSpO2 154
GcsEDArrScoreTimeSinceInj 155
GcsEDArrAssmtCond 156
GcsEDArrScore 163
GcsEDArrPupils 168
GcsEDArrPupilReactR 170
GcsEDArrPupilReactL 172


In [49]:
Counter(fulldata.iloc[:,152])

Counter({'Spontaneous': 517, 'Assisted': 63, nan: 6})

In [30]:
# one-hot encoding oc
gose1['GOSE_Reponder6M_Patient.alone'] = np.zeros(28)
gose1['GOSE_Reponder6M_Patient.plus.relative'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

/Users/ahyeon/anaconda3/envs/activ/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ahyeon/anaconda3/envs/activ/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
# one-hot encoding bm

gose1['cause_Assault'] = [0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0]
gose1['cause_Fall'] = [1,1,1,1,1,1,0,1,1,0,1,0,1,1,1,1,1,1,0,1,1,1,1,0,0,0,1,0]
gose1['cause_MV.car.'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1]
gose1['cause_MV.cyclist.ped.'] = [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]
gose1['cause_MV.motorcycle.'] = np.zeros(28)
gose1['cause_Other'] = np.zeros(28)
gose1['cause_Struck.By'] = np.zeros(28)
gose1['DispER_Discharge.Home'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
gose1['DispER_ICU.Admit'] = [0,1,0,1,0,1,0,1,1,1,1,0,1,0,0,1,1,0,1,1,1,0,1,0,0,0,0,0]
gose1['DispER_Operating.Room'] = [0,0,1,0,1,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0]
gose1['DispER_Stepdown.Admit'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0]
gose1['DispER_Ward Admit'] = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0]
gose1['PatientTypeCoded_ED.Discharge'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
gose1['PatientTypeCoded_ICU.Admit'] = [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0]
gose1['PatientTypeCoded_Ward.Admit'] = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0]
gose1['PresArrivalMethod_Ambulance'] = [1,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1]
gose1['PresArrivalMethod_Helicopter'] = [0,1,0,1,0,0,1,1,1,1,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0]
gose1['PresArrivalMethod_Walk.in'] = np.zeros(28)
gose1['RACE_3CAT_Black'] = np.zeros(28)
gose1['RACE_3CAT_Other'] = np.zeros(28)
gose1['RACE_3CAT_White'] = np.ones(28)
gose1['injurytype_Closed'] = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1]
gose1['injurytype_Penetrating'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]

/Users/ahyeon/anaconda3/envs/activ/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ahyeon/anaconda3/envs/activ/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/ahyeon/anaconda3/envs/activ/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [32]:
keep_cols = np.asarray(list(set(sub_features).intersection(gose1.columns)))
gose1_oc_features = np.asarray(list(set(keep_cols).intersection(oc_features)))
gose1_bm_features = np.asarray(list(set(keep_cols).intersection(bm_features)))

In [33]:
len(gose1_oc_features), len(gose1_bm_features)

(177, 235)

In [34]:
drop_cols = list(set(gose1.columns).difference(keep_cols))

In [35]:
gose1.columns.shape

(943,)

In [36]:
gose1 = gose1.drop(columns = drop_cols)

In [37]:
gose1_bm = gose1.drop(columns = gose1_oc_features)
gose1_oc = gose1.drop(columns = gose1_bm_features)

In [38]:
gose1_bm.shape, gose1_oc.shape

((28, 235), (28, 177))

In [39]:
pd.DataFrame(gose1_bm).to_csv("/Users/ahyeon/Desktop/activ misc/gose1_bm.csv")
pd.DataFrame(gose1_oc).to_csv("/Users/ahyeon/Desktop/activ misc/gose1_oc.csv")

## Masked Rows/Columns

In [180]:
kept_rows = pd.read_csv('/Users/ahyeon/Desktop/TBIanalysis/data/discarded_rows.txt', header=None)
discarded_cols = pd.read_csv('/Users/ahyeon/Desktop/TBIanalysis/data/discarded_cols.txt', header=None)
kept_rows = np.hstack(kept_rows.values)
discarded_cols = np.hstack(discarded_cols.values)
kept_rows.shape, discarded_cols.shape


((247,), (444,))

In [181]:
mask = pd.read_csv('/Users/ahyeon/Desktop/TBIanalysis/data/mask_feature_outcome.txt', header=None)
mask = mask.values
oc_mask = np.where(mask==1)[0]

In [182]:
a = np.arange(0,586)
b = np.arange(0,918)
discarded_rows = list(set(a).difference(kept_rows))
kept_cols = list(set(b).difference(discarded_cols))

In [183]:
len(discarded_rows), len(discarded_cols), len(kept_rows), len(kept_cols)

(339, 444, 247, 474)

In [184]:
dropped_data = data.drop(data.columns[discarded_cols], axis=1)

In [185]:
dropped_data = data.drop(data.index[discarded_rows])

In [186]:
new_oc_mask = list(set(oc_mask).intersection(kept_cols))

In [187]:
data = data.iloc[:, new_oc_mask]
# data = data.drop(data.index[discarded_rows])
data.shape

(586, 216)

In [205]:
temp = data
temp.isnull().sum().sum()/np.product(temp.shape)

0.316820987654321

In [207]:
data = impute(data)

In [3]:
mask = pd.read_csv('/Users/ahyeon/Desktop/TBIanalysis/data/mask_feature_outcome.txt')
mask = mask['0'].values

In [12]:
patients = open('/Users/ahyeon/Desktop/TBIanalysis/data/patient_id.txt').read().split()

In [34]:
outcome = open('/Users/ahyeon/Desktop/TBIanalysis/data/data_matrix_subset_outcome.txt').read()

In [13]:
fulldata.rename( columns={'Unnamed: 0':'id'}, inplace=True )